# データ分析の基礎知識 (2019/10/26 八王子AI moto2g)
## データの準備 ダウンロードしてgoogledriveへコピー
- https://www.kaggle.com/c/titanic/data
- https://vincentarelbundock.github.io/Rdatasets/datasets.html
 - Speed and Stopping Distances of Cars

# データの読み込み

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
# Google Driveへの接続
import google.colab.drive
google.colab.drive.mount('gdrive')

In [0]:
# ファイル読み込み
train = pd.read_csv('gdrive/My Drive/titanic/train.csv')
test = pd.read_csv('gdrive/My Drive/titanic/test.csv')

# データの基礎調査

In [0]:
# 行列数を取得
print(train.shape)
print(test.shape)

## データの中身を見てみる

In [0]:
# 先頭行を表示
train.head()

In [0]:
# 末尾行を表示
test.tail()

In [0]:
# 特定のカラムの内容を表示
train[['Name','Sex','Age']].head(10)

## 統計量を見てみる
- count データ数
- mean 平均
- std 標準偏差
- min 最小値
- 50% 数えていって真ん中の値
- max 最大値


In [0]:
# 統計量を表示
train.describe()

## 欠損値の確認
- セルが欠損値かどうかを返す isnull()
- 列ごとに欠損値の有無を確認する isnull().any()
- 欠損値の数を確認する isnull().sum()

In [0]:
# カラムごとに欠損値の有無を確認
# Age、Cabin、Embarkedに欠損値があることが判明
train.isnull().any()

In [0]:
train.isnull().sum()

## 欠損値への対処
- そのレコードを使用しない
- 他の値にする

### 欠損値のあるレコードを使用しない

In [0]:
# Nanのレコードを削除 dropna()
train.dropna(subset=['Age'])[['Name','Sex','Age']].head(10) # subsetを指定しなければ、どこかのカラムにNaNがレコードが削除される

### 欠損値を他の値にする

In [0]:
# 年齢を平均値で置き換える
train['Age'].mean()

In [0]:
# NaNを他の値にする fillna
train['Age'].fillna(value=train['Age'].mean()).head(10)
# 上記は置換後の値を返すだけなので、実際にこれを使う場合は、データフレームの値を置き換える必要がある
# train['Age'] = train['Age'].fillna(train['Age'].mean())

## NaN以外にも使えないデータはあり得る
- 数値を取るべきカラムに文字
- 数値がありえない範囲になっている
- フォーマットが異なっている

In [0]:
# そのカラムがどういう値をとっているのかを列挙 value_counts()
train['Cabin'].value_counts().head(10)

## ラベルから数値への置換
### 性別を変換

In [0]:
train.head(10)

In [0]:
# 直接返還
train2 = train.copy()
train2['Sex'][train2['Sex'] == 'female'] = 0
train2.head(10)

In [0]:
# 一括で変換
train2 = train.copy()
mapped = {'male':0, 'female':1}
train2['Sex'] = train['Sex'].map(mapped)
train2.head(10)

# グラフを書いてみる

## ヒストグラム
- データの偏りを見る

In [0]:
train['Age'].plot.hist(grid=True, title='histgram')
plt.axvline(x=train['Age'].mean(), color="red")

## 箱ひげ図
- データの範囲を見る
- 線:上下が最大値、最小値
- 箱:上側ヒンジ...大きい方から25%のところにある数値、中央値、下側ヒンジ...大きい方から25%のところにある数値
- 丸:外れ値
- 箱が大きいほどバラけている

In [0]:
# 別のサンプルデータ読み込み 車の速度と停止までの距離
dtrain = pd.read_csv('gdrive/My Drive/dataset/cars_train.csv')
dtest = pd.read_csv('gdrive/My Drive/dataset/cars_test.csv')

In [0]:
dtrain.head(10)

In [0]:
dtrain[['dist', 'speed']].boxplot(by='speed')

## 散布図
- データの相関関係を見る

### 速度と移動距離の相関

In [0]:
# 相関係数 -1 ～ 負の相関 ～ 0:無相関 ～ 正の相関 ～ 1
dtrain[['dist', 'speed']].corr()

In [0]:
dtrain.plot.scatter(x='dist', y='speed')

### 性別と生存の相関 意味無し

In [0]:
# 性別を数値に置き換えたものを別のカラムとして追加
mapped = {'male':0, 'female':1}
train['SexInt'] = train['Sex'].map(mapped)
train[['Sex','SexInt']].head()

In [0]:
train[['SexInt','Survived']].corr()

# 予測してみる
- 説明変数 ... 予測の元になる値
- 目的変数 ... 予測したい値

## 単回帰モデル y = ax + b

In [0]:
from sklearn.linear_model import LinearRegression as LR

In [0]:
# データ読み込み(サンプルデータはcarsを使用)
train = pd.read_csv('gdrive/My Drive/dataset/cars_train.csv')
test = pd.read_csv('gdrive/My Drive/dataset/cars_test.csv')

In [0]:
# 変数にデータを格納
trainX = train["speed"]
y = train["dist"]
testX = test["speed"]

In [0]:
# 単回帰モデルの場合は形式変換が必要(ここはそういうものと覚える)
trainX = trainX.values.reshape(-1,1)
testX = testX.values.reshape(-1,1)

In [0]:
# モデル生成
my_model = LR()
my_model.fit(trainX, y)

In [0]:
# 傾き、切片
(my_model.coef_, my_model.intercept_)

In [0]:
# 1つのテストデータの予測値を見てみる
print("入力{} 予測{}".format(testX[0], testX[0] * my_model.coef_ + my_model.intercept_))

In [0]:
# テストデータ全体の予測値を求める
my_model.predict(testX)

In [0]:
test

In [0]:
# 図示する
# 青:モデル生成に使ったデータ
# 赤:予測モデル
fig = plt.figure()
ax1 = train.plot.scatter(x='speed', y='dist')
x = np.arange(0, 25, 0.1).reshape(-1, 1)
y = my_model.predict(x)
ax1.plot(x, y, 'r--')

## 決定木

In [0]:
from sklearn import tree

In [0]:
# データ読み込み(サンプルデータはtitanicを使用)
train = pd.read_csv('gdrive/My Drive/titanic/train.csv')
test = pd.read_csv('gdrive/My Drive/titanic/test.csv')

In [0]:
# NaNの置き換え
train['Age'] = train['Age'].fillna(value=train['Age'].mean())
# ラベルの数値化 警告出ます
train['Sex'][train['Sex'] == 'male'] = 0
train['Sex'][train['Sex'] == 'female'] = 1

# mappedは指定していない値はNaNになる。下記を2回以上実行するとNaNになるので注意
#mapped = {'male':0, 'female':1}
#train['Sex'] = train['Sex'].map(mapped)

In [0]:
# NaNの置き換え
test['Age'] = test['Age'].fillna(value=test['Age'].mean())
# ラベルの数値化 警告出ます
test['Sex'][test['Sex'] == 'male'] = 0
test['Sex'][test['Sex'] == 'female'] = 1

In [0]:
target = train['Survived'].values
features = train[['Sex','Age']].values 

In [0]:
# 決定木の作成
my_model = tree.DecisionTreeClassifier().fit(features, target)

In [0]:
# テストデータの予測値を求める
testX = test[["Sex", "Age"]].values
my_model.predict(testX)

In [0]:
# kaggleへ提出するCSV作成
pred = my_model.predict(testX)
passenger_id = np.array(test["PassengerId"]).astype(int)
df = pd.DataFrame(data = pred, index = passenger_id, columns = ["Survived"])
df.to_csv("pred.csv", index_label = ["PassengerId"]) 